In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from heptracktool.tools.sparcification_study import SparcifyStudy
from heptracktool.io.pyg_data_reader import TrackGraphDataReader

2025-03-10 23:46:22.504 | WARNING  | heptracktool.tools.utils_graph:<module>:15 - FRNN is not available, install it at https://github.com/murnanedaniel/FRNN. Using PyG radius instead.
2025-03-10 23:46:22.507 | WARNING  | heptracktool.tools.utils_graph:<module>:27 - FAISS is not available, install it at "conda install faiss-gpu -c pytorch" or                     "pip install faiss-gpu". Using PyG radius instead.


In [3]:
file_path = "/global/cfs/cdirs/m3443/usr/xju/graph_sparcification/chep2024_data_feature_store"
reader = TrackGraphDataReader(file_path, name="valset")

valset: Total 1 events in directory: /global/cfs/cdirs/m3443/usr/xju/graph_sparcification/chep2024_data_feature_store


In [18]:
ckpt_name = '/global/cfs/cdirs/m3443/data/GNN4ITK/AcornModels/MetricLearning/best-26489255-f1=0.010138-epoch=479.ckpt'
study = SparcifyStudy(ckpt_name, reader, kval=1000, rval=0.12)

2025-03-10 23:55:28.643 | INFO     | heptracktool.tools.sparcification_study:__init__:38 - # of node features: 37
2025-03-10 23:55:28.643 | INFO     | heptracktool.tools.sparcification_study:__init__:39 - Node features: ['hit_r', 'hit_phi', 'hit_z', 'hit_cluster_x_1', 'hit_cluster_y_1', 'hit_cluster_z_1', 'hit_cluster_x_2', 'hit_cluster_y_2', 'hit_cluster_z_2', 'hit_count_1', 'hit_charge_count_1', 'hit_loc_eta_1', 'hit_loc_phi_1', 'hit_localDir0_1', 'hit_localDir1_1', 'hit_localDir2_1', 'hit_lengthDir0_1', 'hit_lengthDir1_1', 'hit_lengthDir2_1', 'hit_glob_eta_1', 'hit_glob_phi_1', 'hit_eta_angle_1', 'hit_phi_angle_1', 'hit_count_2', 'hit_charge_count_2', 'hit_loc_eta_2', 'hit_loc_phi_2', 'hit_localDir0_2', 'hit_localDir1_2', 'hit_localDir2_2', 'hit_lengthDir0_2', 'hit_lengthDir1_2', 'hit_lengthDir2_2', 'hit_glob_eta_2', 'hit_glob_phi_2', 'hit_eta_angle_2', 'hit_phi_angle_2']
2025-03-10 23:55:28.644 | INFO     | heptracktool.tools.sparcification_study:__init__:40 - Node scales: [1000, 3

In [11]:
input_pyg = reader.pyg_files[0]
print(f"Input PyG: {input_pyg}")
sparse_matrix = study.build_knn_graph(input_pyg)

Input PyG: /global/cfs/cdirs/m3443/usr/xju/graph_sparcification/chep2024_data_feature_store/event000006800-graph.pyg


In [12]:
outfilename = str(input_pyg).replace("pyg", "mtx")
print(f"Output sparse matrix: {outfilename}")
print(f"# of edges: {sparse_matrix.nnz:,}")
study.save_sparse_matrix(sparse_matrix, outfilename)


Output sparse matrix: /global/cfs/cdirs/m3443/usr/xju/graph_sparcification/chep2024_data_feature_store/event000006800-graph.mtx
# of edges: 4,650,364


In [17]:
eval_result = study.eval_sparcification(sparse_matrix)

Number of edges: 4,650,364


2025-03-10 23:53:36.569 | INFO     | heptracktool.tools.edge_perf:eval:9 - True Reco Edges 61,354, True Edges 106,357, Per-edge efficiency: 57.687%
2025-03-10 23:53:36.570 | INFO     | heptracktool.tools.edge_perf:eval:12 - True Edges 106,357, Reco Edges 4,650,364, Per-edge purity: 2.287%
2025-03-10 23:53:48.152 | INFO     | heptracktool.tools.edge_perf:eval:20 - Only 10.47% of true edges are of interests (signal)
2025-03-10 23:53:48.153 | INFO     | heptracktool.tools.edge_perf:eval:21 - True Reco Signal Edges 9,060, True Signal Edges 11,131, Per-edge signal efficiency: 81.394%


In [19]:
eval_result = study.eval_sparcification(sparse_matrix)

Number of edges: 9,300,728


2025-03-10 23:56:14.369 | INFO     | heptracktool.tools.edge_perf:eval:10 - True Reco Edges 155,742, True Edges 212,714, Per-edge efficiency: 73.217%
2025-03-10 23:56:14.371 | INFO     | heptracktool.tools.edge_perf:eval:13 - True Edges 212,714, Reco Edges 9,300,728, Per-edge purity: 2.287%
2025-03-10 23:56:38.020 | INFO     | heptracktool.tools.edge_perf:eval:22 - Only 10.47% of true edges are of interests (signal)
2025-03-10 23:56:38.033 | INFO     | heptracktool.tools.edge_perf:eval:23 - True Reco Signal Edges 22,224, True Signal Edges 22,262, Per-edge signal efficiency: 99.829%


In [14]:
study.reader.data.pt

tensor([ 338.4827, 1834.4453,  392.2282,  ...,  531.9106,  182.8401,
         372.8299], dtype=torch.float64)